In [1]:
import pandas as pd

In [2]:
def print_memory(df):
    print('Memory usage: {0:.2f}'.format(df.memory_usage().sum() / (1024 * 1024)) + ' MB')
    print('{0:,d} x {1:,d}'.format(df.shape[0], df.shape[1]))

## Read file

In [3]:
df = pd.read_csv('data/bronze/bronze.csv')
df = df.iloc[:, 0:13]

print(df.columns)
print('\n')
print_memory(df)

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated'],
      dtype='object')


Memory usage: 79.05 MB
797,046 x 13


## Data cleaning

### not used for now info

In [4]:
dropCols = ['magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'updated']
df = df.drop(dropCols, axis=1)

print_memory(df)

Memory usage: 36.49 MB
797,046 x 6


### weird values

In [5]:
z_0 = df['depth'] < 0

print('z above sup {0:.2%}\n'.format(sum(z_0) / len(df)))

df.loc[z_0, 'depth'] = 0

print_memory(df)

z above sup 1.34%

Memory usage: 36.49 MB
797,046 x 6


## Data parsing

### date and time

In [6]:
date_col = 'time'
datetimes = df[date_col].str.split('T', expand = True)
dates = pd.to_datetime(datetimes.loc[:, 0], format = '%Y-%m-%d')
hours = datetimes.loc[:, 1].str.split(':', expand = True).iloc[:, 0]
hours = hours.astype('int')
df = pd.concat((df, hours.rename('hour')), axis=1)
df = pd.concat((df, dates.rename('date')), axis=1)
df = df.drop(date_col, axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797046 entries, 0 to 797045
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   latitude   797046 non-null  float64       
 1   longitude  797046 non-null  float64       
 2   depth      797041 non-null  float64       
 3   mag        797046 non-null  float64       
 4   id         797046 non-null  object        
 5   hour       797046 non-null  int64         
 6   date       797046 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 42.6+ MB
None


## Output

In [7]:
df.sort_values(['date', 'hour'], ascending = [True, True]).to_csv('data/silver/silver.csv', index = False)